## 문제 3-1 :  콘텐츠분쟁해결 RAG 시스템 - 간단 실습 가이드
### 콘텐츠분쟁해결 사례집을 활용한 법률 자문 RAG 시스템을 순차적으로 구축하는 실습입니다.

## 0단계: 문서 로드
### loader = PyPDFLoader('../data/콘텐츠분쟁해결_사례.pdf')

## 1단계: 문서 분할 설정 
### pythontext_splitter = RecursiveCharacterTextSplitter(
###     chunk_size=1500,        #  법률 사례는 1200-1800자 권장
###     chunk_overlap=300,      #  사례 맥락 보존을 위해 200-400자
###     separators=[
###         "\n【사건개요】",     #  법률 문서 섹션 구분자
###         "\n【쟁점사항】",     #  쟁점 부분 구분
###         "\n【처리경위】",     #  처리 과정 구분
###         "\n【처리결과】",     #  결과 부분 구분
###         "\n■", "\n\n", "\n", ".", " ", ""
###     ]
### )

## 2단계: 임베딩 모델 설정 
### pythonembeddings = UpstageEmbeddings(model="solar-embedding-1-large")

## 3단계: 검색기 설정 
### pythonretriever = vectorstore.as_retriever(
###     search_type="similarity",        #  또는 "mmr" (다양성 고려시)
###     search_kwargs={"k": 5}          #  상위 5개 관련 사례 검색
### )

## 4단계: LLM 설정 
### pythonllm = ChatUpstage(
###         model="solar-pro",
###         base_url="https://api.upstage.ai/v1",
###         temperature=0.2
###     )

## 5단계: 법률 자문 프롬프트 작성 
### pythonprompt_template = """
### 당신은 콘텐츠 분야 전문 법률 자문가입니다. 
### 아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

### 관련 분쟁사례:
### {context}

### 상담 내용: {question}

### 답변 가이드라인:
### 1. 제시된 사례들을 근거로 답변하세요                    # 사례 기반 답변
### 2. 관련 법령이나 조항이 있다면 명시하세요               # 법적 근거 제시
### 3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요    # 판례 참조
### 4. 실무적 해결방안을 단계별로 제시하세요               #  실무 가이드
### 5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요  #  한계 인정

### 전문 법률 조언:"""

## 6단계: QA 체인 생성 
### pythonqa_chain = RetrievalQA.from_chain_type(
###     llm=llm,                        #  위에서 설정한 LLM
###     chain_type="stuff",             #  문서들을 하나로 합쳐서 처리
###     retriever=retriever,            #  위에서 설정한 검색기
###     chain_type_kwargs={"prompt": prompt},  #  법률 자문 프롬프트
###     return_source_documents=True    #  참조 문서도 함께 반환
### )

## 7단계: 테스트 질문 작성 
### pythontest_questions = [
###     "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
###     "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
###     "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
###     "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
###     "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
### ]

## 8단계: 분쟁 유형 분류 함수 ( 선택 )
### def classify_dispute_type(query):
###     game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
###     elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
###     web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]
    
###     query_lower = query.lower()
    
###     if any(keyword in query_lower for keyword in game_keywords):
###         return "게임"
###     elif any(keyword in query_lower for keyword in elearning_keywords):
###         return "이러닝"
###     elif any(keyword in query_lower for keyword in web_keywords):
###         return "웹콘텐츠"
###     else:
###         return "기타"

### 프롬프트 기본 템플릿:
### python# 기본 법률 자문 프롬프트 (5분 만에 완성)
### prompt_template = """당신은 콘텐츠 분야 전문 법률 자문사입니다.

### 관련 분쟁사례: {context}
### 상담 내용: {question}

## 답변 가이드라인:
### 1. 사례를 근거로 답변하세요.
### 2. 관련 법령을 명시하세요.  
### 3. 단계별 해결방안을 제시하세요.
### 4. 유사 사례를 참조하세요.
### 5. 없는 정보는 "확인할 수 없습니다"라고 하세요.

In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain_core.prompts import PromptTemplate

In [2]:
load_dotenv()

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[30:])

O6


In [3]:
USE_UPSTAGE = False
embeddings = None
llm = None

try:
    from langchain_upstage import UpstageEmbeddings, ChatUpstage
    upstage_key = os.getenv("UPSTAGE_API_KEY")
    if not upstage_key:
        raise RuntimeError("UPSTAGE_API_KEY not set")
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large", api_key=upstage_key)
    llm = ChatUpstage(model="solar-pro", api_key=upstage_key, base_url="https://api.upstage.ai/v1", temperature=0.2)
    USE_UPSTAGE = True
    print("Upstage 임베딩,LLM 사용")
except Exception as e:
    print(f"Upstage 사용 불가 -> 폴백: HF 임베딩 + Groq LLM ({e})")
    from langchain_huggingface import HuggingFaceEmbeddings
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    from langchain_openai import ChatOpenAI
    groq_key = os.getenv("GROQ_API_KEY") or os.getenv("OPENAI_API_KEY")
    llm = ChatOpenAI(api_key=groq_key, base_url="https://api.groq.com/openai/v1", model="openai/gpt-oss-120b", temperature=0.2)
    print("HF 임베딩 + Groq LLM 사용")

Upstage 임베딩,LLM 사용


In [4]:
PDF_NAME = "콘텐츠분쟁해결_사례.pdf"
candidate = Path("../data") / PDF_NAME
pdf_path = None

if candidate.is_file():
    pdf_path = candidate.resolve()
else:
    hits = list(Path.cwd().resolve().rglob(PDF_NAME))
    if hits:
        pdf_path = hits[0].resolve()

if not pdf_path:
    raise FileNotFoundError("콘텐츠분쟁해결_사례.pdf 파일을 찾을 수 없습니다. 경로를 확인하세요.")

print("[PDF]", pdf_path)

loader = PyPDFLoader(str(pdf_path))
docs = loader.load()
print("총 페이지 수:", len(docs))

[PDF] C:\mylangchain\mylangchain-app\src\mylangchain_app\data\콘텐츠분쟁해결_사례.pdf
총 페이지 수: 109


In [5]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=300,
    separators=[
        "\n【사건개요】",
        "\n【쟁점사항】",
        "\n【처리경위】",
        "\n【처리결과】",
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)
splits = text_splitter.split_documents(docs)
print("생성된 청크 수:", len(splits))
if not splits:
    raise RuntimeError("분할된 청크가 없습니다. PDF 또는 분할 설정을 확인하세요.")

생성된 청크 수: 104


In [8]:
PERSIST_DIR = str((Path.cwd() / "chroma_law_db").resolve())
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory=PERSIST_DIR,
    collection_name="kocdac_cases"
)

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)
print("벡터DB 준비 완료 →", PERSIST_DIR)

벡터DB 준비 완료 → C:\mylangchain\mylangchain-app\src\mylangchain_app\Practice\chroma_law_db


In [9]:
prompt_text = """너는 콘텐츠 분야 전문 법률 자문가입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요
2. 관련 법령이나 조항이 있다면 명시하세요
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요
4. 실무적 해결방안을 단계별로 제시하세요
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요

전문 법률 조언:"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_text
)

In [10]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)
print("QA 체인 준비 완료")

QA 체인 준비 완료


In [11]:

test_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]

for i, q in enumerate(test_questions, 1):
    print("\n" + "="*80)
    print(f"[Q{i}] {q}")
    result = qa_chain.invoke({"query": q})
    print("\n[답변]\n", result["result"].strip())
    print("\n[참조 출처]")
    for idx, d in enumerate(result.get("source_documents", []), 1):
        meta = d.metadata or {}
        page = meta.get("page", meta.get("source", "N/A"))
        print(f"  - {idx}) page={page}, source={meta.get('source', 'PDF')}")


[Q1] 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?

[답변]
 ### 전문 법률 조언: 온라인 게임 시스템 오류로 인한 아이템 손실 시 법적 대응 방안  

#### 1. **사례 분석 및 쟁점 정리**  
제시된 사례(2006년, 2009년 시스템 오류 사례)에 따르면, 게임회사는 다음 사유로 복구를 거부한 것으로 확인됩니다:  
- **계정 명의 불일치**: 게임회사가 현금거래 및 계정 공유를 금지하는 약관을 근거로, 실소유주라도 명의자가 아닌 경우 복구를 거부(2006년 사례).  
- **시스템 오류 입증 부족**: 다른 이용자에게 동일한 오류가 보고되지 않았거나, 이용 내역상 문제가 없어 게임회사의 과실을 인정하기 어려운 경우(2009년 사례).  

#### 2. **관련 법령 및 약관 검토**  
- **전자상거래법 제17조(계약 해제·해지 및 대금 환급)**  
  - 게임회사가 서비스 제공 과정에서 발생한 시스템 오류로 인해 이용자가 손해를 입은 경우, 민법상 **채무불이행** 또는 **불법행위** 책임(민법 제390조, 제750조)이 발생할 수 있습니다.  
  - 다만, 게임회사가 **"약관에서 시스템 오류로 인한 책임을 명시적으로 배제"**한 경우, 해당 약관이 **약관의 규제에 관한 법률 제6조(불공정약관 무효)**에 따라 무효인지 여부가 쟁점이 됩니다.  
    - 예: "모든 시스템 오류에 대한 책임을 지지 않는다"는 조항은 불공정할 수 있음.  

- **게임산업진흥에 관한 법률 제12조(게임물의 내용 및 운영 기준)**  
  - 게임회사는 이용자에게 **공정한 서비스 제공 의무**가 있으며, 시스템 오류 시 합리적인 조치를 취해야 합니다.  

#### 3. **실무적 해결 방안 (단계별 접근)**  
**(1) 증거 수집 및 보존**  
- **시스템 오류 증거**: 게임 내 로그, 스크린샷, 고객센터 문의 기록, 오류 발생 시점의 접속 기록 등을 